<a href="https://colab.research.google.com/github/ZainabSMU/GenAI/blob/main/GenAI_FINAL_RECENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the main libraries for the project
!pip install -q datasets transformers accelerate peft bitsandbytes sentencepiece faiss-cpu evaluate beautifulsoup4 lxml

import os, math, random, requests
from dataclasses import dataclass

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from datasets import load_dataset, Dataset as HFDataset
from transformers import (
    AutoTokenizer,
    BertConfig,
    BertModel,
    get_cosine_schedule_with_warmup,
    AutoModelForCausalLM,
)

import evaluate
import faiss
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using the following device:", device)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
Using device: cuda


In [ ]:
# Loading the general-purpose corpus from BookCorpus
book_ds = load_dataset("rojagtap/bookcorpus", split="train")

# Keeping only the text column and clean it a bit
def clean_book(example):
    return {"text": example["text"].strip()}

book_ds = book_ds.map(clean_book, remove_columns=book_ds.column_names)

# Filtering out very short/noisy  lines
book_ds = book_ds.filter(lambda e: len(e["text"]) > 50)

# Subsampling to make training feasible
book_ds = book_ds.shuffle(seed=42).select(range(20000))
print("BookCorpus subset size:", len(book_ds))
print(book_ds[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

books_large_p1.txt:   0%|          | 0.00/2.52G [00:00<?, ?B/s]

books_large_p2.txt:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/74004228 [00:00<?, ? examples/s]

Map:   0%|          | 0/74004228 [00:00<?, ? examples/s]

In [ ]:
# Scraping the Paul Graham (PG) essays from the website: paulgraham.com
BASE_URL = "http://www.paulgraham.com/articles.html"

def fetch_pg_essays():
    # Loading the index page listing the articles
    index_html = requests.get(BASE_URL).text
    soup = BeautifulSoup(index_html, "lxml")

    links = []
    # Collecting all the relative .html links that look like essays
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.endswith(".html") and not href.startswith("http"):
            links.append("http://www.paulgraham.com/" + href)

    texts = []
    for url in sorted(set(links)):
        try:
            html = requests.get(url).text
            s = BeautifulSoup(html, "lxml")
            # Since essays are usually inside main <table>, fall back to whole page
            body = s.find("table")
            if body is None:
                body = s
            text = body.get_text(separator=" ", strip=True)
            texts.append({"url": url, "text": text})
        except Exception as e:
            print("Error fetching", url, e)
    return texts

pg_texts = fetch_pg_essays()
print("Fetched essays:", len(pg_texts), "Example URL:", pg_texts[0]["url"])